In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

In [3]:
today = date.today()
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-10-16
yesterday: 2025-10-15


In [5]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [9]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.0f}','beta':'{:,.2f}',
    }

In [11]:
def show_buy(const, name):
    sql = """
    SELECT * 
    FROM buy 
    WHERE name = '{}'
    """.format(name)
    
    df_buy = pd.read_sql(sql, const)
    df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
    df_buy.rename(columns={'volbuy':'shares'},inplace=True)
    df_buy['shares'] = df_buy['shares'].astype('int64')
    return df_buy.style.format(format_dict).hide(axis="index")

In [13]:
def show_dividend(const, name):
    sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
    """.format(name)
    
    df_dividend = pd.read_sql(sql, const)
    df_dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
    df_dividend.columns = df_dividend.columns.str.lower()
    df_dividend['shares'] = df_dividend['shares'].astype('int64')
    df_dividend['xdate'] = pd.to_datetime(df_dividend['xdate'])
    df_dividend['paiddate'] = pd.to_datetime(df_dividend['paiddate'])
    return df_dividend.style.format(format_dict).hide(axis="index")

In [15]:
def show_stocks(conlite, name):
    sql = """
    SELECT * 
    FROM stocks 
    WHERE name = '{}'
    """.format(name)
    
    df_stocks = pd.read_sql(sql, conlite)
    return df_stocks.style.format(format_dict).hide(axis="index")

In [17]:
def update_buy(const, name, new_qty, new_price):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty,
        price = :new_price
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,
        'new_price': new_price,
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [19]:
def update_dividend(const, name, new_qty):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [21]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):  
    print(f"Updating {name} with available_qty = {new_qty} (type: {type(new_qty)})")
    
    # Convert numpy.int64 to native Python int
    new_qty = int(new_qty)
    
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    
    result = conlite.execute(sqlUpd, {
        'new_qty': new_qty,
        'new_unit_cost': new_unit_cost,
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    
    conlite.commit()
    
    # Verify the update worked - wrap the SQL in text()
    verify_sql = text("SELECT available_qty FROM stocks WHERE name = :name")
    verify_result = conlite.execute(verify_sql, {'name': name})
    updated_value = verify_result.fetchone()[0]
    print(f"Value after update: {updated_value} (type: {type(updated_value)})")
    
    return f"Records updated = {result.rowcount}"

In [23]:
def debug_stock(conlite, name):
    # Check the actual data type and value
    sql = """
    SELECT name, available_qty, typeof(available_qty) as type
    FROM stocks 
    WHERE name = ?
    """
    df = pd.read_sql(sql, conlite, params=[name])
    print(f"Data type of available_qty: {df['type'].iloc[0]}")
    print(f"Value of available_qty: {df['available_qty'].iloc[0]}")
    return df

### End of Definition

## Begin of  Buy transaction  

In [27]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
df_buys_latest = pd.read_sql(sql, conpf)

display(df_buys_latest.style.format(format_dict).hide(axis="index"))

name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
PTT,1353,118,2025-10-16,"2,500",31.00,160.43,11.23,Active,"77,671.66",HD,None


In [29]:
transaction = 'B'

names = df_buys_latest['name']
name = names.to_string(index=False)

sr_qty = df_buys_latest['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = df_buys_latest['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

print(transaction, name, qty, buy_price)

B PTT 2500 31.0


### Begin of Update buy table in MySQL database from buy transaction

In [32]:
sql = """
    SELECT *
    FROM buy 
    WHERE name = '{}'
""".format(name)
    
df_buy = pd.read_sql(sql, const)
df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
df_buy.rename(columns={'volbuy':'shares'},inplace=True)
df_buy['shares'] = df_buy['shares'].astype('int64')
df_buy.style.hide(axis="index")

name,date,shares,price,active,period,grade
PTT,2025-08-08,7500,32.000000,1,2,A2


### In case there is the same stock name in port

In [35]:
show_buy(const, name)

name,date,shares,price,active,period,grade
PTT,2025-08-08,"7,500",32.00,1,2,A2


In [37]:
sql = """
    SELECT name, B.* 
    FROM buys B
    JOIN stocks T 
    ON B.stock_id = T.id
    WHERE name = '{}' AND status = 'Active'
    ORDER BY id DESC
""".format(name)
df_buys = pd.read_sql(sql, conpf)
# Format all floats (pandas 1.4.0+)
#display(df_buys.style.format({
#    col: '{:,.2f}' for col in df_buys.select_dtypes(include=float).columns
#}).hide(axis="index"))
df_buys.style.format(format_dict).hide(axis="index")

name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
PTT,1353,118,2025-10-16,"2,500",31.00,160.43,11.23,Active,"77,671.66",HD,None
PTT,1349,118,2025-09-19,"2,500",33.00,170.78,11.95,Active,"82,682.73",HD,None
PTT,1345,118,2025-09-01,"2,500",31.00,160.43,11.23,Active,"77,671.66",HD,None
PTT,1336,118,2025-08-08,"2,500",32.00,165.60,11.59,Active,"80,177.19",HD,None


In [39]:
# Ensure the DataFrame is sorted by id descending and reset index
df_buys = df_buys.sort_values('id', ascending=False).reset_index(drop=True)

# Exclude the first row (latest transaction)
#excluded_df = df_buys.iloc[1:]

# Calculate total price*qty and total qty
total_price_qty = (df_buys['price'] * df_buys['qty']).sum()
total_qty = df_buys['qty'].sum()

# Compute average unit cost
average_unit_cost = total_price_qty / total_qty

print(f"Total qty: {total_qty:,.0f}")
print(f"Average Unit Cost: {average_unit_cost:.2f}")

Total qty: 10,000
Average Unit Cost: 31.75


In [41]:
# Excluding latest row
df_excluding_latest = df_buys[df_buys['id'] != df_buys['id'].max()]
avg_excluding = (df_excluding_latest.eval('qty * price').sum() / 
                 df_excluding_latest['qty'].sum())

# Including all rows
avg_including = (df_buys.eval('qty * price').sum() / df_buys['qty'].sum())

print(f"Average unit cost excluding latest row: {avg_excluding:.2f}")
print(f"Average unit cost including latest row: {avg_including:.2f}")

Average unit cost excluding latest row: 32.00
Average unit cost including latest row: 31.75


In [43]:
new_qty = total_qty
new_unit_cost = avg_including
print(transaction, name, new_qty, new_unit_cost)

B PTT 10000 31.75


In [45]:
update_buy(const, name, new_qty, new_unit_cost)

'Records updated = 1'

In [47]:
show_buy(const, name)

name,date,shares,price,active,period,grade
PTT,2025-08-08,"10,000",31.75,1,2,A2


### In case of totally new stock

In [72]:
if df_buy.empty:
    rcd = {
        'name': 'CPF',
        'date': '2025-08-21',
        'volbuy': 4000,
        'price': 24.1,
        'volsell': 0,
        'volbal': 0,
        'active': 1,
        'dividend': 0,
        'period': 4,
        'grade': 'A1'
    }
    sqlIns = text("""
    INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
    VALUES (:name, :date, :volbuy, :price, :volsell, :volbal, :active, :dividend, :period, :grade)
    """)
    result = const.execute(sqlIns, rcd)
    result.rowcount

In [74]:
show_buy(const, name)

name,date,shares,price,active,period,grade
PTT,2025-08-08,"7,500",32.00,1,2,A2


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [49]:
sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
""".format(name)
    
df_dividend = pd.read_sql(sql, const)
df_dividend.columns = df_dividend.columns.str.lower()
df_dividend['shares'] = df_dividend['shares'].astype('int64')

df_dividend.style.format(format_dict).hide(axis="index")

name,q4,q3,q2,q1,dividend,price,percent,shares,xdate,paiddate,kind,actual
PTT,1.3000,0.0000,0.9000,0.0000,2.2000,0.00,0.00%,"7,500",2025-09-01,2025-09-17,,1


In [51]:
show_dividend(const, name)

name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
PTT,1.3000,0.0000,0.9000,0.0000,2.2000,"7,500",2025-09-01,2025-09-17,,1


In [53]:
new_qty = total_qty
print(name, new_qty)

PTT 10000


In [55]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [57]:
show_dividend(const, name)

name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
PTT,1.3000,0.0000,0.9000,0.0000,2.2000,"10,000",2025-09-01,2025-09-17,,1


### In case of totally new dividend

In [34]:
if df_dividend.empty:
    rcd = {
        'name': 'CPF',
        'q4': '0',
        'q3': '0',
        'q2': '1.00',
        'q1': '0',    
        'dividend': '1.00',      
        'shares': 4000,
        'xdate': '2025-08-29',
        'paiddate': '2025-09-12',
        'kind': ' ',
        'actual': 1
    }
    sqlIns = text("""
    INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
    VALUES (:name, :q4, :q3, :q2, :q1, :dividend, :shares, :xdate, :paiddate, :kind, :actual)
    """)
    print(sqlIns)

In [35]:
if df_dividend.empty:
    result = const.execute(sqlIns, rcd)
    result.rowcount

In [36]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,CPF,0.5500,0.0000,1.0000,0.0000,1.5500,"-10,000",2025-08-29,2025-09-12,,1


### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [59]:
sql = """
    SELECT name, cost, CAST(available_qty AS INTEGER) as available_qty
    FROM stocks
    WHERE name = '{}'
""".format(name)

df_stocks = pd.read_sql(sql, conlite)
df_stocks['available_qty'] = df_stocks['available_qty'].astype('int64')
df_stocks.style.format(format_dict).hide(axis="index")

name,cost,available_qty
PTT,32.00,"7,500"


In [61]:
show_stocks(conlite, name)

id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
151,PTT,35.00,27.00,S,32.00,35.25,0,0.54,32.00,"2,500",-4,4,"7,500",0.000000,0.000000,,SET50


In [65]:
new_buy_target = 30.50
new_sell_target = 31.50
new_buy_qty = 2_500
print(transaction, name, new_qty, buy_price, new_buy_target, new_sell_target, new_buy_qty)

B PTT 10000 31.0 30.5 31.5 2500


In [67]:
update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

Updating PTT with available_qty = 10000 (type: <class 'numpy.int64'>)
Value after update: 10000 (type: <class 'int'>)


'Records updated = 1'

In [69]:
show_stocks(conlite, name)

id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
151,PTT,35.00,27.00,S,30.50,31.50,0,0.54,31.75,"2,500",-4,4,"10,000",0.000000,0.000000,,SET50


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy transaction

### =====================================================

In [65]:
#price_date = '2025-02-14'
sql = """
    SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, 
    FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,
    FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, 
    FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, 
    FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol 
    FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '{}' 
    ORDER BY period, buy.name
""".format(yesterday)
print(sql)


    SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, 
    FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,
    FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, 
    FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, 
    FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol 
    FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2025-09-15' 
    ORDER BY period, buy.name



In [67]:
df_buy = pd.read_sql(sql, const)
df_buy

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,AH,2023-06-08,"1,200",37.00,13.90,"44,400.00","16,680.00","-27,720.00",-62.43
1,1,A1,AWC,2023-06-15,"9,000",4.96,2.42,"44,640.00","21,780.00","-22,860.00",-51.21
2,1,A2,JMART,2023-01-20,"6,800",33.00,10.30,"224,400.00","70,040.00","-154,360.00",-68.79
3,1,C3,JMT,2023-01-26,"4,200",55.50,12.80,"233,100.00","53,760.00","-179,340.00",-76.94
4,1,B2,PTG,2023-07-12,"3,600",11.40,8.50,"41,040.00","30,600.00","-10,440.00",-25.44
5,1,C1,TOA,2023-09-06,"1,000",26.00,14.60,"26,000.00","14,600.00","-11,400.00",-43.85
6,2,A1,AIMIRT,2023-08-17,"17,500",10.60,10.10,"185,500.00","176,750.00","-8,750.00",-4.72
7,2,C1,CPNREIT,2022-08-16,"55,000",18.00,11.90,"990,000.00","654,500.00","-335,500.00",-33.89
8,2,C1,DIF,2020-08-01,"45,000",12.70,9.30,"571,500.00","418,500.00","-153,000.00",-26.77
9,2,C1,GVREIT,2022-08-24,"69,000",7.75,6.40,"534,750.00","441,600.00","-93,150.00",-17.42


In [69]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [71]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\portfolio.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\portfolio.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\portfolio.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\portfolio.csv


In [73]:
df_buy.to_csv(output_file, header=True, index=False)
df_buy.to_csv(icd_file, header=True, index=False)
df_buy.to_csv(god_file, header=True, index=False)
df_buy.to_csv(osd_file, header=True, index=False)

In [75]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)
df_buy.to_excel(xsl_file, index=False)